In [1]:
import sys
sys.path.append('D:/Anaconda/envs/Alive/Lib/site-packages')
import retro
import time
from stable_baselines import PPO2
from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines.common import set_global_seeds
#from OpenAIWrappers
from stable_baselines.common.callbacks import CheckpointCallback
#from wrapper import wrapper
import torch
from torch import nn
from copy import deepcopy

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



d:\Anaconda\envs\Alive\lib\site-packages\stable_baselines\__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [33]:
states = ["ChunLiVsBlanka.1star", "ChunLiVsBalrog.1star", "ChunLiVsBison.1star", "ChunLiVsChunLi.1star", "ChunLiVsDhalsim.1star",
            "ChunLiVsGuille.1star", "ChunLiVsHonda.1star", "ChunLiVsKen.1star", "ChunLiVsRyu.1star", "ChunLiVsSagat.1star", "ChunLiVsVega.1star",
            "ChunLiVsZahgief.1star"]

In [4]:
env = retro.make(game="StreetFighterIISpecialChampionEdition-Genesis", state="ChunLiVsBlanka.1star")
#env = retro.make(game="DonkeyKongCountry2-Snes")
#env = wrapper(env)
#model = PPO2.load("D:/Python/Projects/Hakisa/rl_model_1000000_steps")
#model = PPO2.load("D:/Python/Projects/Hakisa/Donkey_Kong/rl_model_1200000_steps")
obs = env.reset()
total_reward = []
steps = 0
end = False

#model = PPO2(policy="CnnPolicy", env=env, gamma=0.99, n_steps=64, learning_rate=3e-9, vf_coef=0.5, verbose=1)

In [5]:
obs, reward, end, info = env.step(env.action_space.sample())

print(obs.shape)

(200, 256, 3)


In [13]:
print(env.action_space)
print(env.action_space.sample()) # MultiBinary

MultiBinary(12)
[0 1 0 0 0 0 1 1 0 0 0 0]


In [3]:
class Subject(nn.Module):

    def __init__(self, epsilon=0.1):
        # Problem: How to deal with determinism in a Binary Environment (Street Fighter) ?

        super(Subject, self).__init__()

        # Input = 3x200x256

        self.conv1 = nn.Conv2d(3, 16, 3, 1, 1, bias=False)
        self.conv2 = nn.Conv2d(16, 64, 2, 2, 0) # 100x128
        self.conv3 = nn.Conv2d(64, 64, 3, 1, 1)
        self.conv4 = nn.Conv2d(64, 128, 2, 2, 0) # 50x64
        self.conv5 = nn.Conv2d(128, 128, 3, 1, 1)
        self.conv6 = nn.Conv2d(128, 256, 2, 2, 0) # 25x32
        self.conv7 = nn.Conv2d(256, 256, 3, 1, 1)
        self.action = nn.Linear(256*25*32, 12)
        self.reward = nn.Linear((256*25*32)+12+1, 1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, obs, previous_cumulative_reward):

        x = self.conv1(obs)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)

        x = x.view(x.size(0), -1)

        actions = self.action(x)

        x = torch.cat((x, actions, previous_cumulative_reward), -1)

        expected_reward = self.reward(x)

        actions = self.sigmoid(actions)

        del x

        return actions, expected_reward

In [4]:
model = Subject().to(device)

In [18]:
# TRIALS

while steps < 1000:
    obs = torch.from_numpy(obs)
    obs = obs/255
    obs = obs.permute(2, 1, 0).unsqueeze(0).float().to(device)
    env.render()
    action, expected_reward = model(obs)
    action = action.detach().cpu().squeeze(0).numpy()

    teste = []
    for x in action:
        if x > 0.5:
            teste.append(1.)
        else:
            teste.append(0.)

    obs, reward, end, info = env.step(teste)

    steps += 1

    time.sleep(0.005)

env.render(close=True)
env.close()

In [20]:
print(action)
print(teste)
print(expected_reward)

print(reward)
print(end)
print(info)

[0.4976626  0.49443606 0.496801   0.49380603 0.504097   0.49821717
 0.50026226 0.49854857 0.49533316 0.50014585 0.4992451  0.49504086]
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
tensor([[-0.0261]], device='cuda:0', grad_fn=<AddmmBackward0>)
0.0
False
{'enemy_matches_won': 0, 'score': 0, 'matches_won': 0, 'continuetimer': 0, 'enemy_health': 176, 'health': 176}


In [4]:
while steps < 1000:
    env.render()
    #print(obs) # Obs ----> Image [0, 255] ?
    action, state = model.predict(obs)
    obs, reward, end, info = env.step(action)
    steps += 1
    #total_reward.append(reward)
    time.sleep(0.005)

env.close()

print(action)
print(state)
print(reward)
print(end)
print(info)

d:\Anaconda\envs\Alive\lib\site-packages\pyglet\image\codecs\wic.py:434: UserWarning: [WinError -2147417850] Não é possível alterar o modo de thread depois de o mesmo estar definido
  warnings.warn(str(err))


[1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1.]
None
0.0
False
{'enemy_matches_won': 0, 'score': 1200, 'matches_won': 0, 'continuetimer': 0, 'enemy_health': 105, 'health': 176}


: 

In [10]:
reward = torch.zeros((1, 1), device=device) # Cumulative reward
advantage = []
steps = 0
env = retro.make(game="StreetFighterIISpecialChampionEdition-Genesis", state="ChunLiVsBlanka.1star")
obs = env.reset()
obs = torch.from_numpy(obs)
obs = obs/255
obs = obs.permute(2, 1, 0).unsqueeze(0).float().to(device)
save_point = 100 # Also optimization point
previous_predicted_reward = None # BEWARE: this will be the basis for backpropagation (TD-Learning)

model.zero_grad()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-6, eps=1e-8)
# 1e-4 is a common LR. But 1e-6 is also used by RainbowDQN and it's the best one for HierNet
# Note: PPO also uses epsilon = 1e-5, but it seems that eps=1e-7 or 1e-8 are better.

old_policy = deepcopy(model) # For Surrogate Loss. Creating here to reserve some space in memory
old_policy.eval()

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1000, gamma=0.1) # The learning rate should decay linearly until it vanishes

value_criterion = torch.nn.MSELoss()

reward_input = reward.clone() # To avoid issues with inplace operations(optimizer)

In [11]:
while steps < 1000:
    env.render()

    reward_input = reward.clone() # To avoid issues with inplace operations(optimizer)

    if previous_predicted_reward is None:

        _, previous_predicted_reward = model(obs, reward_input)

    action, predicted_reward = model(obs, reward_input)
    action = action.squeeze(0)

    # For Surrogate Loss

    with torch.no_grad():

        previous_action, previous_predicted_reward = old_policy(obs, reward_input)
        previous_action = previous_action.squeeze(0)

    # MultiBinary Environment --> Only 0.0 or 1.0 accepted
    bin = []
    for x in action:
        if x > 0.5:
            bin.append(1.)
        else:
            bin.append(0.)

    obs, scoring, end, info = env.step(bin)
    scoring = torch.tensor(scoring, device=device)

    reward += scoring

    delta = scoring + (0.9995 * predicted_reward.item()) - previous_predicted_reward.item()
    advantage.append(delta)

    for t in reversed(range(len(advantage) - 1)):

        advantage[t] = advantage[t] + 0.9995 * 0.9995 * advantage[t+1]

    # Calculating Entropy - Used to avoid deterministic behavior and it's a possible replace/complement to epsilon-greedy.

    entropy = -(action * torch.log(torch.clamp(action, 1e-10, 1.0))).sum()

    # Since we're dealing with a probability distribution, using exp(log) is more mathmatically correct(and stable)
    # In practice, we're doing a KL-Divergence. To make it less computationally expensive, applying clip.

    action = torch.clamp(action, 1e-10, 1.0)
    previous_action = torch.clamp(previous_action, 1e-10, 1.0)

    ratio = torch.exp(torch.log(action) - torch.log(previous_action))
    clipped_ratio = torch.clamp(ratio, min=0.8, max=1.2)

    surrogate_loss = -torch.mean(torch.minimum(torch.mul(ratio, advantage[-1]), torch.mul(clipped_ratio, advantage[-1])))

    # We can use both the actual cumulative (discounted) reward, or the predicted reward for target. Following Ruo-Ze Liu's code.

    value_loss = value_criterion(previous_predicted_reward, (scoring + 0.9995 * predicted_reward))

    total_loss = surrogate_loss + (value_loss * 0.5) + (entropy * 1e-3)

    total_loss.backward()

    # Performing a single iteration in order to get previous predicted reward for next backpropagation

    obs = torch.from_numpy(obs)
    obs = obs/255
    obs = obs.permute(2, 1, 0).unsqueeze(0).float().to(device)
    _, previous_predicted_reward = model(obs, reward_input)

    steps += 1

    #time.sleep(0.005)

    if steps % save_point == 0:

        old_policy = deepcopy(model)
        old_policy.eval()

        optimizer.step()
        scheduler.step()

        model.zero_grad()

        print(f"Current step: {steps}")
        print(f"Current Loss: {total_loss.item()}")
        print(f"Surrogate Loss: {surrogate_loss.item()}\tValue Loss: {value_loss.item()}\tEntropy: {entropy.item()}\nAdvantage: {advantage[-1].item()}")
        print(bin)
        print(f"Predicted Reward: {predicted_reward.item()}\tCurrent Reward: {reward.item()}\n\n")

        # Performing a single iteration in order to get previous predicted reward for backpropagation

        #obs = torch.from_numpy(obs)
        #obs = obs/255
        #obs = obs.permute(2, 1, 0).unsqueeze(0).float().to(device)

        reward_input = reward.clone()

        _, previous_predicted_reward = model(obs, reward_input)

env.render(close=True)
env.close()

Current step: 100
Current Loss: 0.003941837698221207
Surrogate Loss: -6.768107414245605e-05	Value Loss: 4.5807277970766336e-09	Entropy: 4.00951623916626
Advantage: 6.768107414245605e-05
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Predicted Reward: -0.13536512851715088	Current Reward: 0.0


Current step: 200
Current Loss: 0.006508069112896919
Surrogate Loss: 0.002565478440374136	Value Loss: 6.564538125530817e-06	Entropy: 3.9393084049224854
Advantage: -0.0025621354579925537
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Predicted Reward: -0.22080892324447632	Current Reward: 0.0


Current step: 300
Current Loss: 0.0022563072852790356
Surrogate Loss: -0.0017382482765242457	Value Loss: 3.019752966793021e-06	Entropy: 3.9930453300476074
Advantage: 0.0017377287149429321
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Predicted Reward: -0.15682832896709442	Current Reward: 0.0


Current step: 400
Current Loss: 0.004116264171898365
Surrogate Loss: 0.000

In [27]:
print(obs.shape)
print(bin)

torch.Size([1, 3, 256, 200])
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
